In [1]:
import numpy as np

In [9]:
G = 'B' # Type of classical algebraic group, Input: string 'A','B','C', or 'D'
l = 3 # Rank, Input: positive integer
n = 2*l+1 # Square matrix dimension for type B

In [5]:
"""
Make matrices for the root vectors for the positive roots in type B (i.e., the root vectors in the nilradical).
Store the root vector matrices in a dictionary called rvecs where the keys are the eigenvector names given on page 131
of Erdmann and the values are the eigenvectors as numpy arrays

"mat_type" is the eigenvector matrix form given on page 131 of Erdmann
"indices" is a list of the i,j indices
"matrix" is the eigenvector corresponding to the root of specified type and indices
"coords" are the coordinates of the positive nonzero matrix entry, to be used for the coordinate function

Arguments:

Returns:

"""
def make_rvecs_B(l):
    rvecs = {}

    M = np.zeros((2*l+1,2*l+1),dtype = int)
    # Add root vectors b_i for roots of the form epsilon_i to rvecs
    for i in np.arange(1,l+1,1):
        # b_ij
        M[i,0] = 1
        M[0,l+i] = -1
        hilo = "b" + str(i)
        coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
        coef[i-1:l] = 1
        coef = tuple(coef)
        rvecs.setdefault(coef, {"matrix":M, "coords":[i,0]}) # index 1 \leq i \leq l
        M = np.zeros((2*l+1,2*l+1),dtype = int)
    for i in np.arange(1,l,1):
        for j in np.arange(1,l+1,1):
            # p_ij
            if i<j:
                M[i,l+j] = 1
                M[j,l+i] = -1
                hilo = "p" + str(i) + str(j)
                coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
                coef[i-1:j] = 1
                coef[j-1:l] = 2
                coef = tuple(coef)
                rvecs.setdefault(coef, {"matrix":M, "coords":[i,l+j]}) # indices 1 \leq i < j \leq l
                M = np.zeros((2*l+1,2*l+1),dtype = int)
            # m_ij
            if i != j and i<j:
                M[i,j] = 1
                M[l+j,l+i] = -1
                hilo = "m" + str(i) + str(j)
                coef = np.zeros(l, dtype = int) # Array of length the number of roots in the base
                coef[i-1:j-1] = 1 
                coef = tuple(coef)
                rvecs.setdefault(coef, {"matrix":M, "coords":[i,j]})
                M = np.zeros((2*l+1,2*l+1), dtype = int)
    return rvecs

rvecs = make_rvecs_B(l)
print(rvecs)

{(1, 1, 1): {'matrix': array([[ 0,  0,  0,  0, -1,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]]), 'coords': [1, 0]}, (0, 1, 1): {'matrix': array([[ 0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]]), 'coords': [2, 0]}, (0, 0, 1): {'matrix': array([[ 0,  0,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0]]), 'coords': [3, 0]}, (1, 2, 2): {'matrix': array([[ 0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0

In [24]:
"""
Fix a sequence of the positive roots with a (partial) ordering induced by increasing root height.

Use the Sage function positive_roots_by_height(increasing=True)
"""
def make_root_order(G,l):
    L = RootSystem([G,l]).root_lattice()
    roots = [tuple(root.to_vector()) for root in L.positive_roots_by_height()]
    return roots

roots = make_root_order(G,l)
roots

[(0, 0, 1),
 (0, 1, 0),
 (1, 0, 0),
 (0, 1, 1),
 (1, 1, 0),
 (0, 1, 2),
 (1, 1, 1),
 (1, 1, 2),
 (1, 2, 2)]

In [34]:
"""
Compute the root spaces $\mathfrak{g}_\beta=\mathbb{C}e_\beta$

Notes:
This could be combined with the function that makes the nilradical
The "DeprecationWarning: invalid escape sequence \m" that returns when this cell is run in Jupyter is because of the 
backslash character used in the TeX for this comment
"""
def make_root_spaces(rvecs):
    rspaces = {} # Create a dictionary rspaces with keys the roots as stored in rvecs and values the root spaces k*e_alpha
    num = len(rvecs) # Set the number of iterations to equal the number of positive roots; we'll need this for most things in this function
    syms = np.array([var("r"+str(list(rvecs.keys())[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(num)], dtype = object) # Name some not-necessarily-distinct field elements
    # Note that rvecs[list(rvecs.keys())[1]]['matrix'] extracts the eigenvector matrix from rvecs
    mats = [np.dot(syms[i],rvecs[list(rvecs.keys())[i]]['matrix']) for i in range(num)] # Multiply the root vectors by an arbitrary field element to get the root spaces
    for i in range(num):
        rspaces.setdefault(list(rvecs.keys())[i],mats[i])
    return rspaces

make_root_spaces(rvecs)

<>:5: DeprecationWarning: invalid escape sequence \m
<>:5: DeprecationWarning: invalid escape sequence \m
<>:5: DeprecationWarning: invalid escape sequence \m
<ipython-input-34-b46544783c78>:5: DeprecationWarning: invalid escape sequence \m
  """


{(1,
  1,
  1): array([[0, 0, 0, 0, -r111, 0, 0],
        [r111, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], dtype=object),
 (0,
  1,
  1): array([[0, 0, 0, 0, 0, -r011, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [r011, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], dtype=object),
 (0,
  0,
  1): array([[0, 0, 0, 0, 0, 0, -r001],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [r001, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], dtype=object),
 (1,
  2,
  2): array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, r122, 0],
        [0, 0, 0, 0, -r122, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 

In [35]:
"""
Compute a matrix representing an arbitrary element of the nilradical

Input: rvecs
Returns: numpy.ndarray ; Arbitrary element of the nilradical
"""
def make_nilrad(rvecs):
    rspaces = {} # Create a dictionary rspaces with keys the roots as stored in rvecs and values the root spaces k*e_alpha
    
    num = len(rvecs) # Set the number of iterations to equal the number of positive roots; we'll need this for most things in this function
    syms = np.array([var("n"+str(list(rvecs.keys())[i]).replace("(", "").replace(")", "").replace(" ", "").replace(",", "")) for i in range(num)], dtype = object) # Name some not-necessarily-distinct field elements
    # Note that rvecs[list(rvecs.keys())[1]]['matrix'] extracts the eigenvector matrix from rvecs
    mats = [np.dot(syms[i],rvecs[list(rvecs.keys())[i]]['matrix']) for i in range(num)] # Multiply the root vectors by an arbitrary field element to get the root spaces
    nil_rad = sum(mats) # Set an arbitrary element of the nilradical equal to the sum of the root spaces
    
    for i in range(num):
        rspaces.setdefault(list(rvecs.keys())[i],mats[i])
    
    return nil_rad #,rspaces, mats

nil_rad = make_nilrad(rvecs)
rdict

{(1,
  1,
  1): array([[0, 0, 0, 0, -n111, 0, 0],
        [n111, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], dtype=object),
 (0,
  1,
  1): array([[0, 0, 0, 0, 0, -n011, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [n011, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], dtype=object),
 (0,
  0,
  1): array([[0, 0, 0, 0, 0, 0, -n001],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [n001, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], dtype=object),
 (1,
  2,
  2): array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, n122, 0],
        [0, 0, 0, 0, -n122, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 

In [ ]:
"""
Compute the NT-submodules $\mathfrak{m}_i$
"""

# Scratch

In [145]:
list(rvecs.keys())[1]

(0, 1, 1)

In [161]:
mats[0]

array([[0, 0, 0, 0, -n111, 0, 0],
       [n111, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [162]:
mats[1]

array([[0, 0, 0, 0, 0, -n011, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [n011, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [169]:
(mats[0]+mats[1])@nil_rad-nil_rad@(mats[0]+mats[1])

array([[0, 0, 0, 0, n011*n100, 0, 0],
       [-n011*n100, 0, 0, 0, 0, 0, -n001*n111],
       [0, 0, 0, 0, 0, 0, -n001*n011],
       [0, 0, 0, 0, n001*n111, n001*n011, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [144]:
nil_rad@rvecs[list(rvecs.keys())[1]]['matrix']-rvecs[list(rvecs.keys())[1]]['matrix']@nil_rad

array([[0, 0, 0, 0, -n4, 0, 0],
       [n4, 0, 0, 0, 0, -n0, 0],
       [0, 0, 0, 0, n0, 0, n2],
       [0, 0, 0, 0, 0, -n2, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=object)

In [52]:
g = LieAlgebra(QQ, cartan_type=[G,l]) #, representation="matrix"

In [91]:
B3 = RootSystem(["B",3]).root_space()
roots=list(B3.positive_roots())
roots

[alpha[1],
 alpha[2],
 alpha[3],
 alpha[1] + alpha[2],
 alpha[2] + 2*alpha[3],
 alpha[2] + alpha[3],
 alpha[1] + alpha[2] + 2*alpha[3],
 alpha[1] + alpha[2] + alpha[3],
 alpha[1] + 2*alpha[2] + 2*alpha[3]]

In [53]:
g

Lie algebra of ['B', 3] in the Chevalley basis

In [19]:
g.lie_algebra_generators()

Finite family {alpha[1]: E[alpha[1]], alpha[2]: E[alpha[2]], alpha[3]: E[alpha[3]], -alpha[1]: E[-alpha[1]], -alpha[2]: E[-alpha[2]], -alpha[3]: E[-alpha[3]], alphacheck[1]: h1, alphacheck[2]: h2, alphacheck[3]: h3}

In [20]:
g.inject_variables()

Defining e1, e2, e3, f1, f2, f3, h1, h2, h3


In [94]:
print(e1)

E[alpha[1]]


In [67]:
n = LieAlgebra(QQ, cartan_type=['A',4], nilpotent=True)

In [ ]:
# Fix an enumeration of the positive roots s.t. ht(\beta_i)\leq ht(\beta_{i+1})



In [100]:
so4 = lie_algebras.so(QQ, 5, representation='matrix'); so4
so4.gens()

(
[ 0  1  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]
[ 0  0  0  0  0]  [ 0  0  0  0  2]  [ 1  0  0  0  0]  [ 0  0  0  0  0]
[ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0 -1  0]  [ 0  0  0  0  0]
[ 0  0 -1  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0  0]  [ 0  0  0  0 -1]
[ 0  0  0  0  0], [ 0  0  0 -2  0], [ 0  0  0  0  0], [ 0  1  0  0  0],

[ 1  0  0  0  0]  [ 0  0  0  0  0]
[ 0 -1  0  0  0]  [ 0  2  0  0  0]
[ 0  0 -1  0  0]  [ 0  0  0  0  0]
[ 0  0  0  1  0]  [ 0  0  0 -2  0]
[ 0  0  0  0  0], [ 0  0  0  0  0]
)